In [1]:
import string
import numpy as np
import pandas as pd
def read_file(file_path):
    with open(file_path,'r') as file:
        text=file.read()
    return text


def remove_punctuation(text):
    translator=str.maketrans('','',string.punctuation)
    cleaned_text=text.translate(translator)
    words=cleaned_text.split()
    return words



def generate_k_shingles(words,k):
    k_shingles=[''.join(words[i:i+k]) for i in range(len(words)-k+1)]
    return k_shingles

k=5
file_path1='/content/Backyard cricket.txt'
text1=read_file(file_path1)
words1=remove_punctuation(text1)
shingles1=generate_k_shingles(words1,k)


file_path2='/content/Cricket.txt'
text2=read_file(file_path2)
words2=remove_punctuation(text2)
shingles2=generate_k_shingles(words2,k)

file_path3='/content/Baseball.txt'
text3=read_file(file_path3)
words3=remove_punctuation(text3)
shingles3=generate_k_shingles(words3,k)

all_shingles=shingles1+shingles2+shingles3



# Step 1: Combine all shingles and find unique shingles
unique_shingles = list(set(all_shingles))
print(len(unique_shingles))
# Step 2: Initialize an empty incidence matrix
incidence_matrix = np.zeros((len(unique_shingles), 3))

# Step 3: Populate the incidence matrix
for i, shingle in enumerate(unique_shingles):
    if shingle in shingles1:
        incidence_matrix[i, 0] = 1  # Document 1
    if shingle in shingles2:
        incidence_matrix[i, 1] = 1  # Document 2
    if shingle in shingles3:
        incidence_matrix[i, 2] = 1  # Document 3

# Step 4: Create a DataFrame for better visualization (optional)
incidence_df = pd.DataFrame(incidence_matrix, index=unique_shingles, columns=["Document 1", "Document 2", "Document 3"])
incidence_df.insert(0,"Row",range(0,len(unique_shingles)))
# Print the incidence matrix
print(incidence_df)



353
                                   Row  Document 1  Document 2  Document 3
scoringonerunforeach                 0         0.0         1.0         0.0
Baseballisabatandballsport           1         0.0         0.0         1.0
onafieldatthe                        2         0.0         1.0         0.0
withabatTheobjective                 3         0.0         0.0         1.0
beachcricketcorridorcricketgarden    4         1.0         0.0         0.0
...                                ...         ...         ...         ...
aroundfourbasestoscore             348         0.0         0.0         1.0
subcontinentandboxcricketin        349         1.0         0.0         0.0
withthebatandthen                  350         0.0         1.0         0.0
holdingbatswithoneplayer           351         0.0         1.0         0.0
torunthebaseshaving                352         0.0         0.0         1.0

[353 rows x 4 columns]


In [2]:
#apply Hash Function
incidence_df['h1'] = (incidence_df['Row']+1) % 1000
incidence_df['h2'] = (2 * incidence_df['Row']+5) % 1000

incidence_df = incidence_df.sort_values(by=['h1','h2'])

print(incidence_df)

                                   Row  Document 1  Document 2  Document 3  \
scoringonerunforeach                 0         0.0         1.0         0.0   
Baseballisabatandballsport           1         0.0         0.0         1.0   
onafieldatthe                        2         0.0         1.0         0.0   
withabatTheobjective                 3         0.0         0.0         1.0   
beachcricketcorridorcricketgarden    4         1.0         0.0         0.0   
...                                ...         ...         ...         ...   
aroundfourbasestoscore             348         0.0         0.0         1.0   
subcontinentandboxcricketin        349         1.0         0.0         0.0   
withthebatandthen                  350         0.0         1.0         0.0   
holdingbatswithoneplayer           351         0.0         1.0         0.0   
torunthebaseshaving                352         0.0         0.0         1.0   

                                    h1   h2  
scoringonerunfore

In [3]:
num_hashes = 2
num_docs = 3
signature_matrix = np.full((num_hashes, num_docs), np.inf)
print(signature_matrix)

for i in range(len(unique_shingles)):
    if incidence_df.iloc[i, 1] == 1:  # Document 1
        signature_matrix[0, 0] = min(signature_matrix[0, 0], incidence_df.iloc[i]['h1'])
        signature_matrix[1, 0] = min(signature_matrix[1, 0], incidence_df.iloc[i]['h2'])
    if incidence_df.iloc[i, 2] == 1:  # Document 2
        signature_matrix[0, 1] = min(signature_matrix[0, 1], incidence_df.iloc[i]['h1'])
        signature_matrix[1, 1] = min(signature_matrix[1, 1], incidence_df.iloc[i]['h2'])
    if incidence_df.iloc[i, 3] == 1:  # Document 3
        signature_matrix[0, 2] = min(signature_matrix[0, 2], incidence_df.iloc[i]['h1'])
        signature_matrix[1, 2] = min(signature_matrix[1, 2], incidence_df.iloc[i]['h2'])
print(signature_matrix)

signature_df = pd.DataFrame(signature_matrix, index=['h1', 'h2'], columns=["Document 1", "Document 2", "Document 3"])
print("\nsignature_matrix :\n")
print(signature_df)

[[inf inf inf]
 [inf inf inf]]
[[ 5.  1.  2.]
 [13.  5.  7.]]

signature_matrix :

    Document 1  Document 2  Document 3
h1         5.0         1.0         2.0
h2        13.0         5.0         7.0
